## Committing Changes and Basics of Statement Execution
https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#committing-changes

https://docs.sqlalchemy.org/en/20/tutorial/dbapi_transactions.html#basics-of-statement-execution